In [ ]:
# For Google Colab
# ! pip install pretty_midi
# ! pip install fastai  --upgrade
# ! pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# Getting MAESTRO MIDI dataset. See
# https://magenta.tensorflow.org/datasets/maestro
! wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip

--2020-12-19 16:42:48--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 74.125.137.128, 2607:f8b0:4023:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58416533 (56M) [application/octet-stream]
Saving to: ‘maestro-v3.0.0-midi.zip’

maestro-v3.0.0-midi 100%[===================>]  55.71M  86.2MB/s    in 0.6s    

2020-12-19 16:42:49 (86.2 MB/s) - ‘maestro-v3.0.0-midi.zip’ saved [58416533/58416533]



In [ ]:
import pretty_midi
import zipfile
import tarfile
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from fastai.vision.all import Image

In [ ]:
with zipfile.ZipFile('maestro-v3.0.0-midi.zip', 'r') as zf:
  zf.extractall()

In [ ]:
frequency = 4
midi_length = 100
midi_time = midi_length / frequency
data_timestep = frequency * 5
cut = frequency * 3

In [ ]:
midi_time

7.5

# Invistagete the data and get a subset for training

In [ ]:
df = pd.read_csv('maestro-v3.0.0/maestro-v3.0.0.csv')  

In [ ]:
df.head(10)

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.wav,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.wav,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.wav,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.wav,397.857508
5,Alexander Scriabin,"5 Preludes, Op.15",validation,2009,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.midi,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.wav,400.557826
6,Alexander Scriabin,"Entragete, Op.63",test,2009,2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi,2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.wav,163.745830
7,Alexander Scriabin,"Etude Op. 2 No.1; Etudes Op. 8, Nos. 5, 11 and 12",train,2013,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.wav,563.904351
8,Alexander Scriabin,"Etude Op. 42, Nos. 4 & 5",test,2009,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.wav,136.315302
9,Alexander Scriabin,"Etude Op. 8, No. 13",validation,2009,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.midi,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.wav,167.085837


In [ ]:
df['duration'].describe()

count    1276.000000
mean      560.463595
std       443.057431
min        45.155208
25%       261.981576
50%       429.168241
75%       685.018817
max      2624.663508
Name: duration, dtype: float64

In [ ]:
c_df = df.groupby(['canonical_title'])['midi_filename'].count()

In [ ]:
c_df.where(c_df>1).describe()

count    204.000000
mean       3.068627
std        2.088013
min        2.000000
25%        2.000000
50%        2.000000
75%        3.000000
max       18.000000
Name: midi_filename, dtype: float64

In [ ]:
# About a half of our data is consists of midi records of the same compositions what is not good for education process
c_df.where(c_df>1).sum()

626.0

In [ ]:
# Let's deduplicate our data
train_data = df.drop_duplicates(subset = 'canonical_title', keep = 'first')[['midi_filename', 'duration']]

In [ ]:
train_data.head(10)

,midi_filename,duration
0,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi,698.661160
3,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi,872.640588
4,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi,397.857508
5,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.midi,400.557826
6,2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi,163.745830
7,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi,563.904351
8,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi,136.315302
9,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.midi,167.085837
10,2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1-D6_09_Track09_wav.midi,102.007110
11,2006/MIDI-Unprocessed_19_R1_2006_01-07_ORIG_MID--AUDIO_19_R1_2006_04_Track04_wav.midi,550.997045


In [ ]:
train_data.describe()

,duration
count,854.000000
mean,551.704522
std,442.581746
min,45.155208
25%,254.777056
50%,398.347625
75%,686.354214
max,2622.724407


In [ ]:
# A total time duration of all midi files
train_data['duration'].sum()

471155.6615315958

# Create a traing dataset

The dataset will contain piano rolls *$midi_time* sec length each.
We will try to avoid including the begging and ending parts of the original rolls while splitting it.

In [ ]:
! rm -rf data
! mkdir data

In [ ]:
max_v = np.zeros(128)
for _, row in tqdm(train_data.iterrows(), total=train_data.shape[0]):
  midi_pretty_format = pretty_midi.PrettyMIDI('maestro-v3.0.0/' + row['midi_filename'])
  piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
  piano_roll = piano_midi.get_piano_roll(fs = frequency)
  max_v = np.maximum(max_v, piano_roll.max(axis = 1))

used_channels = max_v > 0

In [ ]:
max_v

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 121.,
       118., 109., 108., 110., 113., 118., 120., 114., 116., 119., 115.,
       115., 119., 120., 119., 123., 119., 119., 121., 124., 122., 121.,
       123., 121., 126., 122., 123., 126., 120., 122., 121., 122., 119.,
       124., 121., 124., 117., 123., 119., 119., 118., 123., 120., 117.,
       117., 118., 118., 119., 122., 125., 115., 126., 116., 126., 123.,
       126., 126., 125., 126., 121., 121., 118., 117., 122., 116., 122.,
       121., 118., 126., 121., 119., 122., 121., 119., 192., 121., 115.,
       177., 120., 204., 125., 123., 116., 112., 125., 106., 106.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.])

We 
do not want to take into account channels which are never used

In [ ]:
used_channels

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]

In [ ]:
first_ch = np.where(used_channels==True)[0][0]
last_ch = np.where(used_channels==True)[0][-1]
channels = len(np.where(used_channels==True)[0])
channels, first_ch, last_ch

(88, 21, 108)

In [ ]:
for _, row in tqdm(train_data.iterrows(), total=train_data.shape[0]):
  midi_pretty_format = pretty_midi.PrettyMIDI('maestro-v3.0.0/' + row['midi_filename'])
  piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
  piano_roll = piano_midi.get_piano_roll(fs = frequency)[used_channels, cut:-cut]

  start = 0
  end = midi_length
  while end < piano_roll.shape[1]:
    #print(end, piano_roll.shape[1])
    one_hot = str()
    for i in range(0, channels):
      if piano_roll[i, end] > 0:
        one_hot += '1'
      else:
        one_hot += '0'
    piano_roll_part = piano_roll[:, start: end]
    piano_roll_part[piano_roll_part > 0] = 255
    img = Image.fromarray(np.uint8(piano_roll_part), 'L')
    img.save('data/' + row['midi_filename'].replace('/', '_') + '_[{},{}]___{}.png'.format(start, end, one_hot))

    start += data_timestep
    end += data_timestep

In [ ]:
! ls data | wc -l

In [ ]:
# For Google Colab
#with tarfile.open('data.tar.gz', 'w:gz') as tf:
#  tf.add('data')